In [2]:
import sys
!pip3 install matplotlib
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow
!pip3 install import-ipynb
!pip3 install cuda-python
!pip3 install -U scikit-learn
!pip3 install bayesian-optimization

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import math
import pickle
import time

import import_ipynb
import AttackClassificationModel

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers.legacy import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.1 MB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.0/155.0 kB 3.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 3.1 MB/s eta 0:00:0000:01
  Created wheel for jax: filename=jax-0.4.8-py3-none-any.whl size=1439678 sha256=f22012c2d9579c17cc82de4ac1642fc63feacbfe71890b0037659836d6d2a565
  Stored in directory: /home/andressa.amaral/.cache/pip/wheels/7a/cc/68/76b3636744b4f749a4704db1b57abdf07c769304a500f843bf
Successfully built jax
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uni


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

# Data Pre-Processing

In [ ]:
df_mirai = AttackClassificationModel.get_files_nbaiot_mirai('../../nbaiot/Danmini_Doorbell/mirai')
df_gafgyt = AttackClassificationModel.get_files_nbaiot_gafgyt('../../nbaiot/Danmini_Doorbell/gafgyt')

# Mirai Ack
df_mirai_ack, label_mirai_ack = AttackClassificationModel.get_attack(0, df_mirai)

# Mirai Scan
df_mirai_scan, label_mirai_scan = AttackClassificationModel.get_attack(1, df_mirai)

# Mirai Syn
df_mirai_syn, label_mirai_syn = AttackClassificationModel.get_attack(2, df_mirai)

# Mirai Udp
df_mirai_udp, label_mirai_udp = AttackClassificationModel.get_attack(3, df_mirai)

# Mirai Udpplain
df_mirai_udpplain, label_mirai_udpplain = AttackClassificationModel.get_attack(4, df_mirai)

# Gafgyt Combo
df_gafgyt_combo, label_gafgyt_combo = AttackClassificationModel.get_attack(5, df_gafgyt)

# Gafgyt Junk
df_gafgyt_junk, label_gafgyt_junk = AttackClassificationModel.get_attack(6, df_gafgyt)

# Gafgyt Scan
df_gafgyt_scan, label_gafgyt_scan = AttackClassificationModel.get_attack(7, df_gafgyt)

# Gafgyt Tcp
df_gafgyt_tcp, label_gafgyt_tcp = AttackClassificationModel.get_attack(8, df_gafgyt)

# Gafgyt Udp
df_gafgyt_udp, label_gafgyt_udp = AttackClassificationModel.get_attack(9, df_gafgyt)

In [ ]:
# Normalize with the min-max scaler
scaler = MinMaxScaler()
df_mirai_ack_norm = scaler.fit_transform(df_mirai_ack)
df_mirai_scan_norm = scaler.fit_transform(df_mirai_scan)
df_mirai_syn_norm = scaler.fit_transform(df_mirai_syn)
df_mirai_udp_norm = scaler.fit_transform(df_mirai_udp)
df_mirai_udpplain_norm = scaler.fit_transform(df_mirai_udpplain)
df_gafgyt_combo_norm = scaler.fit_transform(df_gafgyt_combo)
df_gafgyt_junk_norm = scaler.fit_transform(df_gafgyt_junk)
df_gafgyt_scan_norm = scaler.fit_transform(df_gafgyt_scan)
df_gafgyt_tcp_norm = scaler.fit_transform(df_gafgyt_tcp)
df_gafgyt_udp_norm = scaler.fit_transform(df_gafgyt_udp)

# Sets

In [ ]:
# Train
len_mirai_ack_train = int(0.7 * len(df_mirai_ack_norm))
X_mirai_ack_train = df_mirai_ack_norm[:len_mirai_ack_train]

len_mirai_scan_train = int(0.7 * len(df_mirai_scan_norm))
X_mirai_scan_train = df_mirai_scan_norm[:len_mirai_scan_train]

len_mirai_syn_train = int(0.7 * len(df_mirai_syn_norm))
X_mirai_syn_train = df_mirai_syn_norm[:len_mirai_syn_train]

len_mirai_udp_train = int(0.7 * len(df_mirai_udp_norm))
X_mirai_udp_train = df_mirai_udp_norm[:len_mirai_udp_train]

len_mirai_udpplain_train = int(0.7 * len(df_mirai_udpplain_norm))
X_mirai_udpplain_train = df_mirai_udpplain_norm[:len_mirai_udpplain_train]

len_gafgyt_combo_train = int(0.7 * len(df_gafgyt_combo_norm))
X_gafgyt_combo_train = df_gafgyt_combo_norm[:len_gafgyt_combo_train]

len_gafgyt_junk_train = int(0.7 * len(df_gafgyt_junk_norm))
X_gafgyt_junk_train = df_gafgyt_junk_norm[:len_gafgyt_junk_train]

len_gafgyt_scan_train = int(0.7 * len(df_gafgyt_scan_norm))
X_gafgyt_scan_train = df_gafgyt_scan_norm[:len_gafgyt_scan_train]

len_gafgyt_tcp_train = int(0.7 * len(df_gafgyt_tcp_norm))
X_gafgyt_tcp_train = df_gafgyt_tcp_norm[:len_gafgyt_tcp_train]

len_gafgyt_udp_train = int(0.7 * len(df_gafgyt_udp_norm))
X_gafgyt_udp_train = df_gafgyt_udp_norm[:len_gafgyt_udp_train]

X_train = np.concatenate([X_mirai_ack_train, X_mirai_scan_train, X_mirai_syn_train, X_mirai_udp_train, X_mirai_udpplain_train,
                          X_gafgyt_combo_train, X_gafgyt_junk_train, X_gafgyt_scan_train, X_gafgyt_tcp_train, X_gafgyt_udp_train])
Y_train = np.concatenate([label_mirai_ack[:len_mirai_ack_train], label_mirai_scan[:len_mirai_scan_train], label_mirai_syn[:len_mirai_syn_train], 
                          label_mirai_udp[:len_mirai_udp_train], label_mirai_udpplain[:len_mirai_udpplain_train],  
                          label_gafgyt_combo[:len_gafgyt_combo_train], label_gafgyt_junk[:len_gafgyt_junk_train], 
                          label_gafgyt_scan[:len_gafgyt_scan_train], label_gafgyt_tcp[:len_gafgyt_tcp_train], label_gafgyt_udp[:len_gafgyt_udp_train]])

In [ ]:
# Test
len_mirai_ack_test = len_mirai_ack_train + int(0.15 * len(df_mirai_ack_norm))
X_mirai_ack_test = df_mirai_ack_norm[len_mirai_ack_train : len_mirai_ack_test]

len_mirai_scan_test = len_mirai_scan_train + int(0.15 * len(df_mirai_scan_norm))
X_mirai_scan_test = df_mirai_scan_norm[len_mirai_scan_train : len_mirai_scan_test]

len_mirai_syn_test = len_mirai_syn_train + int(0.15 * len(df_mirai_syn_norm))
X_mirai_syn_test = df_mirai_syn_norm[len_mirai_syn_train : len_mirai_syn_test]

len_mirai_udp_test = len_mirai_udp_train + int(0.15 * len(df_mirai_udp_norm))
X_mirai_udp_test = df_mirai_udp_norm[len_mirai_udp_train : len_mirai_udp_test]

len_mirai_udpplain_test = len_mirai_udpplain_train + int(0.15 * len(df_mirai_udpplain_norm))
X_mirai_udpplain_test = df_mirai_udpplain_norm[len_mirai_udpplain_train : len_mirai_udpplain_test]

len_gafgyt_combo_test = len_gafgyt_combo_train + int(0.15 * len(df_gafgyt_combo_norm))
X_gafgyt_combo_test = df_gafgyt_combo_norm[len_gafgyt_combo_train : len_gafgyt_combo_test]

len_gafgyt_junk_test = len_gafgyt_junk_train + int(0.15 * len(df_gafgyt_junk_norm))
X_gafgyt_junk_test = df_gafgyt_junk_norm[len_gafgyt_junk_train : len_gafgyt_junk_test]

len_gafgyt_scan_test = len_gafgyt_scan_train + int(0.15 * len(df_gafgyt_scan_norm))
X_gafgyt_scan_test = df_gafgyt_scan_norm[len_gafgyt_scan_train : len_gafgyt_scan_test]

len_gafgyt_tcp_test = len_gafgyt_tcp_train + int(0.15 * len(df_gafgyt_tcp_norm))
X_gafgyt_tcp_test = df_gafgyt_tcp_norm[len_gafgyt_tcp_train : len_gafgyt_tcp_test]

len_gafgyt_udp_test = len_gafgyt_udp_train + int(0.15 * len(df_gafgyt_udp_norm))
X_gafgyt_udp_test = df_gafgyt_udp_norm[len_gafgyt_udp_train : len_gafgyt_udp_test]

X_test = np.concatenate([X_mirai_ack_test, X_mirai_scan_test, X_mirai_syn_test, X_mirai_udp_test, X_mirai_udpplain_test,
                          X_gafgyt_combo_test, X_gafgyt_junk_test, X_gafgyt_scan_test, X_gafgyt_tcp_test, X_gafgyt_udp_test])
Y_test = np.concatenate([label_mirai_ack[len_mirai_ack_train : len_mirai_ack_test], label_mirai_scan[len_mirai_scan_train : len_mirai_scan_test], label_mirai_syn[len_mirai_syn_train : len_mirai_syn_test], 
                         label_mirai_udp[len_mirai_udp_train : len_mirai_udp_test], label_mirai_udpplain[len_mirai_udpplain_train : len_mirai_udpplain_test],  
                         label_gafgyt_combo[len_gafgyt_combo_train : len_gafgyt_combo_test], label_gafgyt_junk[len_gafgyt_junk_train : len_gafgyt_junk_test], 
                         label_gafgyt_scan[len_gafgyt_scan_train : len_gafgyt_scan_test], label_gafgyt_tcp[len_gafgyt_tcp_train : len_gafgyt_tcp_test], label_gafgyt_udp[len_gafgyt_udp_train : len_gafgyt_udp_test]])


In [ ]:
# Validation
X_mirai_ack_val = df_mirai_ack_norm[len_mirai_ack_test:]
X_mirai_scan_val = df_mirai_scan_norm[len_mirai_scan_test:]
X_mirai_syn_val = df_mirai_syn_norm[len_mirai_syn_test:]
X_mirai_udp_val = df_mirai_udp_norm[len_mirai_udp_test:]
X_mirai_udpplain_val = df_mirai_udpplain_norm[len_mirai_udpplain_test:]
X_gafgyt_combo_val = df_gafgyt_combo_norm[len_gafgyt_combo_test:]
X_gafgyt_junk_val = df_gafgyt_junk_norm[len_gafgyt_junk_test:]
X_gafgyt_scan_val = df_gafgyt_scan_norm[len_gafgyt_scan_test:]
X_gafgyt_tcp_val = df_gafgyt_tcp_norm[len_gafgyt_tcp_test:]
X_gafgyt_udp_val = df_gafgyt_udp_norm[len_gafgyt_udp_test:]

X_val = np.concatenate([X_mirai_ack_val, X_mirai_scan_val, X_mirai_syn_val, X_mirai_udp_val, X_mirai_udpplain_val,
                        X_gafgyt_combo_val, X_gafgyt_junk_val, X_gafgyt_scan_val, X_gafgyt_tcp_val, X_gafgyt_udp_val])
Y_val = np.concatenate([label_mirai_ack[len_mirai_ack_test:], label_mirai_scan[len_mirai_scan_test:], label_mirai_syn[len_mirai_syn_test:], 
                        label_mirai_udp[len_mirai_udp_test:], label_mirai_udpplain[len_mirai_udpplain_test:],  
                        label_gafgyt_combo[len_gafgyt_combo_test:], label_gafgyt_junk[len_gafgyt_junk_test:], 
                        label_gafgyt_scan[len_gafgyt_scan_test:], label_gafgyt_tcp[len_gafgyt_tcp_test:], label_gafgyt_udp[len_gafgyt_udp_test:]])

In [ ]:
X_train_CNN = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_CNN = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val_CNN = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

samples, feature, depth = X_train_CNN.shape

# Model

In [ ]:
# Initial values
batch_size = 50
learning_rate = 0.008
epochs = 5

# Dictionary
dict_params = { 'learning_rate': learning_rate, 'batch_size': round(batch_size), 'epochs': round(epochs) }
pbounds = { 'learning_rate': (0.000001, 0.001), 'batch_size': (10, 23), 'epochs': (1, 100) }

In [ ]:
cnn_model = AttackClassificationModel.CNN(feature, depth, 10)
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

# Hyperparameter Optimization

In [ ]:
def maximize_training(X_train = X_train_CNN,
                      Y_train = Y_train, 
                      X_val = X_val_CNN, 
                      Y_val = Y_val, 
                      X_test = X_test_CNN, 
                      Y_test = Y_test, 
                      learning_rate = learning_rate, 
                      epochs = epochs, 
                      batch_size = batch_size,
                      reduce_lr = reduce_lr):
    
    nadam = Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
    model = AttackClassificationModel.CNN_nbaiot(feature, depth)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
    
    history = model.fit(X_train, Y_train, 
                        epochs = dict_params['epochs'], 
                        batch_size = dict_params['batch_size'], 
                        validation_data = (X_val, Y_val),
                        callbacks = [reduce_lr],
                        verbose = 0)

    scores = model.evaluate(X_test, Y_test)
    return scores[1]

In [ ]:
# Apply Bayesian optimization to choose the best hyperparameters

opt = BayesianOptimization(f = maximize_training,
                           pbounds = pbounds,
                           verbose = 2, 
                           random_state = 1)

opt_start = time.time()

opt.maximize(init_points = 5, n_iter = 5)

opt_end = time.time()
opt_time = opt_end - opt_start
print("Optimization time:", opt_time)

learning_rate = opt.max['params']['learning_rate']
epochs = round(opt.max['params']['epochs'])
batch_size = round(opt.max['params']['batch_size'])


# Classify Attacks

In [ ]:
# Hyperparameters
n = 5

In [ ]:
nadam = Nadam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
cnn_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])

for i in range(n):
    print("Iteration " + str(i))
    
    # Train
    cnn_model, train_time = AttackClassificationModel.train(X_train_CNN, Y_train, X_val_CNN, Y_val, epochs, batch_size, reduce_lr, cnn_model)
    
    # Test
    Y_pred, test_time = AttackClassificationModel.test(X_test_CNN, cnn_model)

    # Multi classification metrics
    labels = ['Mirai_Ack','Mirai_Scan','Mirai_Syn','Mirai_Udp','Mirai_Udpplain', 'Bashlite_Combo','Bashlite_Junk','Bashlite_Scan','Bashlite_Udp', 'Bashlite_Tcp']
    acc, f1, pre, rec = AttackClassificationModel.get_scores(Y_test, Y_pred, labels)
    
    # Print results
    AttackClassificationModel.print_results(learning_rate,
                                            epochs,
                                            batch_size,
                                            X_train,
                                            X_val,
                                            X_test,
                                            opt_time,
                                            train_time,
                                            test_time,
                                            acc,
                                            f1,
                                            pre,
                                            rec,
                                            Y_test,
                                            Y_pred,
                                            "CNN",
                                            "Results/danmini_doorbell.txt")
    
    print("")

In [ ]:
conf_matrix = confusion_matrix(Y_test, Y_pred)

fig, ax = plt.subplots(figsize = (6, 6))
ax.matshow(conf_matrix, cmap = plt.cm.Blues, alpha = 0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x = j, y = i,s = conf_matrix[i, j], va = 'center', ha = 'center', size = 'large')
 
plt.xlabel('Predictions', fontsize=12)
plt.ylabel('Actuals', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
plt.savefig("Results/Confusion Matrix/DD CNN")
plt.show()